In [2]:
import yfinance as yf

In [1]:
import os
os.getcwd()

'/Volumes/MishaPersonal/Files/code/fintech/canprefs'

In [3]:
# Hello, this is a stock db test
import pandas as pd
import sqlite3

In [2]:
df = pd.read_pickle("./pwf_pr_t.av")

In [5]:
# create a sql connection
con = sqlite3.connect('stock.db')
c = con.cursor()



In [ ]:
# create price table
query1 = """CREATE TABLE IF NOT EXISTS prices (
Date TEXT NOT NULL,
ticker TEXT NOT NULL,
price REAL,
PRIMARY KEY(Date, ticker)
)"""
c.execute(query1.replace('\n',' '))
# create volume table
query2 = """CREATE TABLE IF NOT EXISTS volume (
Date TEXT NOT NULL,
ticker TEXT NOT NULL,
volume REAL,
PRIMARY KEY(Date, ticker)
)"""
c.execute(query2.replace('\n',' '))

In [4]:
# backtest inputs
bt_inputs = {'tickers': ['BA', 'UNH', 'MCD', 'HD'],
'start_date': '2019-01-01',
'end_date': '2021-06-01'}

In [8]:
def download(bt_inputs, proxy = None):
    data = yf.download(tickers= bt_inputs['tickers'],
                       start = bt_inputs['start_date'],   
                       end = bt_inputs['end_date'],
                       interval = '1d',
                       prepost = True,
                       threads = True,
                       proxy = proxy)
    return data

In [9]:
test = download(bt_inputs)

[*********************100%***********************]  4 of 4 completed


In [10]:
adj_close = test['Adj Close']
volume = test['Volume']
adj_close.head()

,BA,HD,MCD,UNH
Date,,,,
2019-01-02,314.645142,162.099030,165.785156,233.873474
2019-01-03,302.100555,158.526291,164.692841,227.495728
2019-01-04,317.822601,163.236664,167.875580,230.156311
2019-01-07,318.823395,166.452118,169.702393,230.598145
2019-01-08,330.891937,167.251282,170.060226,233.681335


In [15]:
# convert wide to long
adj_close_long = pd.melt(adj_close.reset_index(), id_vars='Date', 
                         value_vars=bt_inputs['tickers'], 
                         var_name ="ticker", value_name="price")
volume_long = pd.melt(volume.reset_index(), id_vars='Date', 
                      value_vars=bt_inputs['tickers'], 
                      var_name = "ticker", value_name = "volume")


In [16]:
adj_close_long

,Date,ticker,price
0,2019-01-02,BA,314.645142
1,2019-01-03,BA,302.100555
2,2019-01-04,BA,317.822601
3,2019-01-07,BA,318.823395
4,2019-01-08,BA,330.891937
...,...,...,...
2423,2021-05-24,HD,313.197662
2424,2021-05-25,HD,315.097687
2425,2021-05-26,HD,315.674652
2426,2021-05-27,HD,317.107147


In [17]:
adj_close_long.to_sql('prices', con, if_exists='append', index=False)
volume_long.to_sql('volume', con, if_exists='append', index=False)

In [5]:
con = sqlite3.connect('./stock.db')
c = con.cursor()

In [6]:
# inputs
select_tickers = bt_inputs['tickers']
start_date = bt_inputs['start_date']
end_date = bt_inputs['end_date']
# construct query


query = """
select * from prices
where ticker in ('"""+ "','".join(select_tickers) + """')
and Date >= '"""+ start_date + """'
and Date < '""" + end_date + "'"

c.execute(query.replace('\n',' '))
result = pd.DataFrame(c.fetchall(), columns = ['Date', 'ticker', 'price'])
# convert to datetime
result['Date'] = pd.to_datetime(result['Date'])

In [7]:
result

,Date,ticker,price
0,2019-01-02,BA,314.645142
1,2019-01-02,HD,162.099030
2,2019-01-02,MCD,165.785156
3,2019-01-02,UNH,233.873474
4,2019-01-03,BA,302.100555
...,...,...,...
2423,2021-05-27,UNH,411.194000
2424,2021-05-28,BA,247.020004
2425,2021-05-28,HD,317.246429
2426,2021-05-28,MCD,233.889999
